# Assignment 3

In [1]:
import pandas as pd
import numpy as np

# Load MovieLens 100K dataset into a dataframe of pandas
names = ['user_id', 'item_id', 'rating', 'timestamp']

#Change the filepath to your data
df = pd.read_csv('C:/Users/shrey/Desktop/ml-100k/u.data', sep='\t', names=names)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [2]:
# Select 500 most active users and 500 most active items from the dataset
n_most_active_users = 500
n_most_active_items = 500

user_ids = df.groupby('user_id').count().sort_values(by='rating', ascending=False).head(n_most_active_users).index
item_ids = df.groupby('item_id').count().sort_values(by='rating', ascending=False).head(n_most_active_items).index
df = df[(df['user_id'].isin(user_ids)) & (df['item_id'].isin(item_ids))]

In [3]:
# Map new internal ID for items
i_ids = df['item_id'].unique().tolist()
item_dict = dict(zip(i_ids, [i for i in range(len(i_ids))]))
df['item_id'] = df['item_id'].map(item_dict)

# Split Dataset

In [4]:
# The number of training users and active users
n_training_users = 300
n_active_users = n_most_active_users - n_training_users

# The number of GIVEN ratings for active users
GIVEN = 20

# Randomly select users from the most active users as training set
random_uids = np.random.choice(df.user_id.unique(), n_training_users, replace=False)
train_df = df[df['user_id'].isin(random_uids)]
# Map new internal ID for all users in the training set
u_ids = train_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
train_df['user_id'] = train_df['user_id'].map(user_dict)

# The rest of users are active users for testing
remain_df = df[~df['user_id'].isin(random_uids)]
# Map new internal ID for all active users
u_ids = remain_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
remain_df['user_id'] = remain_df['user_id'].map(user_dict)

# Randomly select GIVEN ratings for active users
active_df = remain_df.groupby('user_id').sample(n=GIVEN, random_state=1024)

test_df = remain_df[~remain_df.index.isin(active_df.index)]

D:\_System_Folder\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\_System_Folder\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
# Convert the format of datasets to matrices
df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_training_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_training_users), 'rating': 0})
train_ds = df_zeros.merge(train_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_active_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_active_users), 'rating': 0})
active_ds = df_zeros.merge(active_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')
test_ds = df_zeros.merge(test_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

train_ds, active_ds, test_ds

(item_id  0    1    2    3    4    5    6    7    8    9    ...  490  491  492  \
 user_id                                                    ...                  
 0        0.0  0.0  4.0  4.0  4.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
 1        4.0  0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 2        4.0  0.0  5.0  0.0  1.0  0.0  3.0  2.0  0.0  0.0  ...  0.0  0.0  0.0   
 3        0.0  0.0  0.0  5.0  4.0  4.0  0.0  0.0  0.0  5.0  ...  0.0  0.0  0.0   
 4        0.0  4.0  4.0  4.0  3.0  2.0  3.0  0.0  0.0  1.0  ...  0.0  0.0  0.0   
 ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 295      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 296      0.0  0.0  5.0  0.0  0.0  0.0  4.0  0.0  0.0  4.0  ...  0.0  0.0  0.0   
 297      4.0  0.0  4.0  0.0  0.0  4.0  3.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
 298      0.0  0.0  5.0  4.0  0.0  1.0  0.0  0.0  0.0  1.0  ...  0.0  0.0  0.0   
 299      0.0  0

In [7]:

imputed_train_ds = train_ds.values.copy()
print(imputed_train_ds.shape)

(300, 500)


# The Prediction code

In [ ]:

LAMBDA = 0.7    # λ
GAMMA = 10      # γ
DELTA = 10      # δ
ITA = 0.7       # η
THETA = 0.7     # θ
EPSILON = 1e-9

def sim(r1, r2):
    
    RA = r1 - np.mean(r1)
    RU = r2 - np.mean(r2)
   
    sim = (np.dot(RA,RU))/(np.sqrt(np.sum(np.square(RA)))*np.sqrt(np.sum(np.square(RU))))
    return sim

def significance(r1,r2, item_user):
    c = 0
    simScore = sim(r1,r2)
    # if its items then item_user = 1 or if users then item_user = 0
    
    if item_user == 0:
        
        for i in range(0,len(r1)):
            if r1[i] != 0 and r2[i] != 0:
                c = c+1
        sig = (min(c, LAMBDA)/ LAMBDA) * simScore
    else:
        
        for i in range(0,len(r1)):
            if r1[i] != 0 and r2[i] != 0:
                c = c+1
                
        sig = (min(c, DELTA)/ DELTA) * simScore
    
    return sig

def simNeighbour(data_no, t, item_user):
    
    S_list = []
    sig_list = []
    if item_user == 0: # if users
        for i in range(0,t.shape[0]):
            sig = significance(t[data_no,:], t[i,:], item_user)
            sig_list.append(sig)
            
            if sig > ITA:
                S_list.append(i)
    
    
    if item_user == 1: # if items
        for i in range(0,t.shape[1]):
            sig = significance(t[:,data_no], t[:,j], item_user)
            sig_list.append(sig)
            
            if sig > THETA:
                S_list.append(i)
    
    return S_list,sig_list


for i in range(0,imputed_train_ds.shape[0]):
    for j in range(0, imputed_train_ds.shape[1] ):
        if imputed_train_ds[i,j] == 0:
            S_u, sig_list_user = simNeighbour(i, imputed_train_ds, 0)
            S_i, sig_list_item = simNeighbour(j, imputed_train_ds, 1)
          
            
            if S_u:
                if S_i:

                    r = LAMBDA

                if not S_i:

                    r = 1

            if not S_u:

                if S_i:
                    r = 0

            sim_ua = np.asarray(sig_list_user)
            r_ua = imputed_train_ds[:,j] - np.mean(imputed_train_ds[i,:])
            
            sim_ik = np.asarray(sig_list_item)
            r_ik = imputed_train_ds[i,:] - np.mean(imputed_train_ds[:,j])
            
            u_ = np.mean(imputed_train_ds[i,:])
            i_ = np.mean(imputed_train_ds[:,j])
            
            U = u_ + np.sum(np.dot(sim_ua, r_ua))/np.sum(sim_ua)
            I = i_ + np.sum(np.matmul(sim_ik, r_ik))/np.sum(sim_ik)
            
            imputed_train_ds[i,j] = (r * U) + ((1-r) * I)

            if not S_u and not S_i:
                U = np.mean(imputed_train_ds[i,:])
                I = np.mean(imputed_train_ds[:,j])
                
                imputed_train_ds[i,j] = (r * U) + ((1-r) * I)
                
print(imputed_train_ds)

# Evaluation

### Compute Pearson Correlation Coefficient of All Pairs of Items between active set and imputed training set

In [ ]:
imputed_train_ds = pd.DataFrame(imputed_train_ds)
imputed_train_ds

In [ ]:
active_user_pearson_corr = np.zeros((active_ds.shape[0], train_ds.shape[0]))

# Compute Pearson Correlation Coefficient of All Pairs of Users between active set and imputed training set
for i, user_i_vec in enumerate(active_ds.values):
    for j, user_j_vec in enumerate(imputed_train_ds.values):
        
        # ratings corated by the current pair od users
        mask_i = user_i_vec > 0
        mask_j = user_j_vec > 0

        # corrated item index, skip if there are no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        if len(corrated_index) == 0:
            continue

        # average value of user_i_vec and user_j_vec
        mean_user_i = np.sum(user_i_vec) / (np.sum(np.clip(user_i_vec, 0, 1)) + EPSILON)
        mean_user_j = np.sum(user_j_vec) / (np.sum(np.clip(user_j_vec, 0, 1)) + EPSILON)

        # compute pearson corr
        user_i_sub_mean = user_i_vec[corrated_index] - mean_user_i
        user_j_sub_mean = user_j_vec[corrated_index] - mean_user_j

        r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
        r_uj_sub_r_j_sq = np.square(user_j_sub_mean)

        r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
        r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))

        sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = (min(len(corrated_index), GAMMA) / GAMMA) * sim

        active_user_pearson_corr[i][j] = weighted_sim

active_user_pearson_corr

## Predict Ratings of Testing Set

In [ ]:
K = 10

test_ds_pred = np.zeros_like(test_ds.values)

for (i, j), rating in np.ndenumerate(test_ds.values):

    if rating > 0:

        sim_user_ids = np.argsort(active_user_pearson_corr[i])[-1:-(K + 1):-1]

        #==================user-based==================#
        # the coefficient values of similar users
        sim_val = active_user_pearson_corr[i][sim_user_ids]

        # the average value of the current user's ratings
        sim_users = imputed_train_ds.values[sim_user_ids]
        user_mean = np.sum(active_ds.values[i]) / (np.sum(np.clip(active_ds.values[i], 0, 1)) + EPSILON)
        sim_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)

        # select the users who rated item j
        mask_rated_j = sim_users[:, j] > 0
        
        # sim(u, v) * (r_vj - mean_v)
        sim_r_sum_mean = sim_val[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])
        
        user_based_pred = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val[mask_rated_j]) + EPSILON)
        user_based_pred = np.clip(user_based_pred, 0, 5)

        test_ds_pred[i][j] = user_based_pred
        
test_ds_pred


## Compute MAE and RMSE

In [ ]:
# MAE
MAE = np.sum(np.abs(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1))

# RMSE
RMSE = np.sqrt(np.sum(np.square(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1)))

print("MAE: {}, RMSE: {}" .format(MAE, RMSE))